In [1]:
import cv2
from statistics import mode
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [2]:
face_detection = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
emotion_classifier = load_model('CustomResnet.h5')

In [4]:
emotion_input_size = (48,48)
emotion_offsets = (20,40)
frame_window = 10

In [5]:
emotion_window = []

In [6]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).
emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [7]:
def apply_offsets(face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)

In [8]:
def normalise_image(gray_img):
    return gray_img/255.0

In [9]:
def draw_bounding_box(face_coordinates, image_array, color):
    x, y, w, h = face_coordinates
    cv2.rectangle(image_array, (x, y), (x + w, y + h),color ,thickness=2)

In [10]:
def draw_text(coordinates, image_array, text, color, x_offset=0, y_offset=0,
                                                font_scale=2, thickness=2):
    x, y = coordinates[:2]
    cv2.putText(image_array, text, (x + x_offset, y + y_offset),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)

In [ ]:
cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)
while True:
    bgr_image = video_capture.read()[1]
    gray_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)
    faces = face_detection.detectMultiScale(gray_image,1.3,5)
    
    for face_coordinates in faces:
        x1,x2,y1,y2 = apply_offsets(face_coordinates,emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face,(emotion_input_size))
        except:
            continue
            
        gray_face = normalise_image(gray_face)
        gray_face = np.expand_dims(gray_face,0)
        gray_face = np.expand_dims(gray_face,-1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_label = emotions[emotion_label_arg]
        emotion_window.append(emotion_label)
        
        if(len(emotion_window))> frame_window:
           emotion_window.pop(0)
        try:
           emotion_mode = mode(emotion_window)
        except:
           continue
           
        draw_bounding_box(face_coordinates,rgb_image,(0,255,0))
        draw_text(face_coordinates,rgb_image,emotion_mode,(255,255,0),0,-45,1,1)
    
    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
           
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
           
video_capture.release()
cv2.destroyAllWindows()
    

In [12]:
bgr_image = cv2.imread('test3.jpg')

In [13]:
cv2.namedWindow('window_frame')
gray_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2GRAY)
rgb_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)
faces = face_detection.detectMultiScale(gray_image,1.3,5)
for face_coordinates in faces:
    x1,x2,y1,y2 = apply_offsets(face_coordinates,emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]
    try:
        gray_face = cv2.resize(gray_face,(emotion_input_size))
    except:
        continue

    gray_face = normalise_image(gray_face)
    gray_face = np.expand_dims(gray_face,0)
    gray_face = np.expand_dims(gray_face,-1)
    emotion_prediction = emotion_classifier.predict(gray_face)
    emotion_probability = np.max(emotion_prediction)
    emotion_label_arg = np.argmax(emotion_prediction)
    emotion_label = emotions[emotion_label_arg]
#     emotion_window.append(emotion_label)

#     if(len(emotion_window))> frame_window:
#        emotion_window.pop(0)
#     try:
#        emotion_mode = mode(emotion_window)
#     except:
#        continue

    draw_bounding_box(face_coordinates,rgb_image,(0,255,0))
    draw_text(face_coordinates,rgb_image,emotion_label,(255,255,0),0,-45,1,1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)

    if cv2.waitKey(10000) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [4]:
from resnets_utils import *
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [5]:
import pandas as pd
dataset = pd.read_csv('28577_36420_bundle_archive/fer2013.csv')
test_set = dataset[dataset['Usage']=='PrivateTest']
test_set = test_set.reset_index(drop=True)
X_test,Y_test = test_set['pixels'],test_set['emotion']
X_test2 = np.zeros((X_test.shape[0],48*48)) 
for i in range(X_test2.shape[0]):
    p = X_test[i].split(' ')
    for j in range(X_test2.shape[1]):
        X_test2[i][j] = int(p[j])
Y_test2 = np.array(Y_test)
X_test3 = X_test2.reshape(X_test2.shape[0],48,48)
Y_test = Y_test2
X_test = X_test3

X_test = X_test/255.0
Y_test = convert_to_one_hot(Y_test, 7).T
X_test = X_test.reshape(X_test.shape[0],48,48,1)
emotion_classifier = load_model('CustomResnet.h5')
preds = emotion_classifier.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3589/3589 [==============================] - 1s 415us/step
Loss = 1.033392020414785
Test Accuracy = 0.6288659572601318


In [9]:
emotion_classifier.get_layer('bn_conv1').uses_learning_phase

AttributeError: 'BatchNormalization' object has no attribute 'uses_learning_phase'